In [4]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from xgboost import XGBClassifier
from scipy.optimize import curve_fit
import seaborn as sns
from other import data_preprocessing
import xgboost

In [47]:
file_name = "model/xgb.pkl"
xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [57]:
datapath = 'data/test.parquet'
test = pd.read_parquet(datapath)

In [58]:
test_new = test
test_new['None_count'] = test['values'].apply(data_preprocessing.none_count)
test_new['Len'] = test['values'].apply(len)
test_emp = test_new.loc[test_new['None_count'] > 0]
test_new = test_new.loc[test_new['None_count'] == 0].drop(['None_count'], axis=1)
len(test_emp), (test_emp['None_count'] / test_emp['Len']).min()

(21, 1.0)

In [59]:
test_new = data_preprocessing.create_new_features(test_new)
test_new.head()

19979it [00:24, 824.88it/s]


,id,dates,values,Len,Max,Mean,Sum,Var,Inflow,Outflow,Static_count,Trend,Bias,A,D
0,6125,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[1.85, -0.04, 0.19, -0.45, -0.75, -0.95, -2.91...",57,3.13,0.037895,2.160000,2.267992,38.91,41.030000,20,-0.010018,0.318403,-0.112889,-0.000026
1,26781,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[-0.41, 0.39, -0.47, -0.9, -1.46, -0.51, 0.51,...",55,3.85,-0.016909,-0.930000,1.561658,38.71,35.920000,24,-0.009304,0.234312,-0.660982,0.000041
2,13333,"[2016-06-01T00:00:00.000000000, 2016-07-01T00:...","[-0.29, -1.26, 0.17, -1.22, 0.45, -0.94, 0.16,...",55,3.46,-0.039569,-2.176292,1.001027,25.46,23.782191,32,0.025446,-0.726617,-0.133820,-0.000063
3,53218,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[-1.47, 1.55, -0.03, 0.57, -0.57, 0.6, 0.27, 1...",78,3.43,0.159231,12.420000,1.724264,47.50,46.710000,37,-0.008521,0.487274,-0.284269,0.000009
4,84204,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[2.33, 1.39, -1.03, -2.64, 1.89, 1.77, 1.43, 1...",65,2.57,-0.094705,-6.155851,1.545744,34.84,38.667317,28,-0.024730,0.696643,1.321546,-0.000044


In [60]:
subm_pred = xgb_model_loaded.predict_proba(test_new[['Sum','D','Trend','A','Bias','Mean','Static_count','Var','Outflow','Max','Len','Inflow']])[:,1]

In [64]:
test_new['score'] = subm_pred
test['score'] = 0.6
test.loc[test.index.isin(test_new.index), 'score'] = test_new['score']

In [65]:
test[['id', 'score']].to_csv('submission.csv')